In [1]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import scipy
import numpy as np

## Short-time fourier transform

In [2]:
def stft(data, fs, window_length_ms=30, window_step_ms=20, windowing_function=None):
    window_length = int(window_length_ms*fs/1000)
    window_step = int(window_step_ms*fs/1000)
    if windowing_function == None:
        windowing_function = np.sin(np.pi*np.arange(0.5, window_length, 1)/window_length)**2 # Hann windowing function

    total_length = len(data)
    window_count = int((total_length - window_length)/window_step) + 1

    spectrum_length = int(window_length/2) + 1
    spectrogram = np.zeros((window_count, spectrum_length))

    for k in range(window_count):
        starting_position = k*window_step

        data_vector = data[starting_position:(starting_position+window_length), ]
        window_spectrum = np.abs(scipy.fft.rfft(data_vector*windowing_function, n=window_length))

        spectrogram[k, :] = window_spectrum
    return spectrogram

In [3]:
# def istft(data, fs, window_length_ms=30, window_step_ms=20):
#     window_length = int(window_length_ms*fs/1000)
#     window_step = int(window_step_ms*fs/1000)
#     windowing_fn = np.sin(np.pi*np.arange(0.5, window_length, 1)/window_length)**2

#     total_length = len(data)
#     window_count = int((total_length-window_length)/window_step) + 1
    
#     spectrum_length = int(window_length/2) + 1 # FFT output is symmetric -> we only need half
#     spectrogram = np.zeros((window_count, spectrum_length))

#     for k in range(window_count):
#         starting_position = k*window_step

#         data_vector = data[starting_position:(starting_position+window_length), ]
#         window_spectrum = np.abs(scipy.fft.rfft(data_vector*windowing_fn, n=window_length))

#         spectrogram[k, : ] = window_spectrum

#     return spectrogram

## Zero crossing rate

In [4]:
def zcr(data, fs, window_length_ms=30, window_step_ms=20):
    window_length = int(window_length_ms*fs/1000)
    window_step = int(window_step_ms*fs/1000)
    # windowing_fn = np.sin(np.pi*np.arange(0.5, window_length, 1)/window_length)

    total_length = len(data)
    window_count = int((total_length-window_length)/window_step) + 1

    y = np.zeros(window_count)
    for k in range(window_count):
        starting_position = k*window_step
        data_vector = data[starting_position:(starting_position+window_length), ]
        y[k] = np.sum(np.abs(np.diff(np.sign(data_vector))))

    return y

## This is mel scale with the standard O'Shaughnessy formula

In [5]:
def freq2mel(f):
    return 2595 * np.log10(1 + (f/700))

def mel2freq(mel):
    return 700 * (10**(mel/2595) - 1)

In [6]:
# Convert a frequency spectrum (linear Hz) to Mel spectrogram (Mel scale)
def melfilterbank(speclen, maxfreq, melbands=20):
    maxmel = freq2mel(maxfreq) # convert max frequency (nyquish) to mel
    mel_idx = np.array(np.arange(0.5, melbands, 1)/melbands)*maxmel
    freq_idx = mel2freq(mel_idx)

    melfilterbank = np.zeros((speclen, melbands))
    freqvec = np.arange(0, speclen, 1)*maxfreq/speclen
    
    for k in range(melbands-2):
        if k>0:
            upslope = (freqvec-freq_idx[k])/(freq_idx[k+1]-freq_idx[k])
        else:
            upslope = 1 + 0*freqvec

        if (k < (melbands-3)):
            downslope = 1 - (freqvec-freq_idx[k+1])/(freq_idx[k+2]-freq_idx[k+1])
        else:
            downslope = 1 + 0*freqvec

        triangle = np.max([0*freqvec, np.min([upslope, downslope], axis=0)], axis=0)
        melfilterbank[:, k] = triangle

    melreconstruct = np.dot(np.diag(np.sum(melfilterbank**2+1e-12, axis=0)**-1), np.transpose(melfilterbank))

    return melfilterbank, melreconstruct

In [ ]:
def linearfilterbank(speclen, maxfreq_Hz, bandwidth_Hz=500):
    bandstep_Hz = bandwidth_Hz/2
    bands = int(maxfreq_Hz/bandstep_Hz) + 1
    filterbank = np.zeros((speclen, bands))
    freqvec = np.arange(0, speclen, 1)*maxfreq_Hz/speclen
    freq_idx = np.arange(-bandstep_Hz/2, maxfreq_Hz+bandstep_Hz/2, bandstep_Hz)
    for k in range(bands-1):
        if k > 0:
            upslope = (freqvec-freq_idx[k])/(freq_idx[k+1]-freq_idx[k])
        else:
            upslope = 1 + 0*freqvec

        if (k < (bands-2)):
            downslope = 1 - (freqvec-freq_idx[k+1])/(freq_idx[k+2]-freq_idx[k+1])
        else:
            downslope = 1 + 0*freqvec

        triangle = np.max([0*freqvec, np.min([upslope, downslope], axis=0)], axis=0)
        filterbank[:, k] = triangle

    reconstruct = np.dot(np.diag(np.sum(filterbank**2+1e-12, axis=0)**-1), np.transpose(filterbank))

    return filterbank, reconstruct